In [ ]:
from pymavlink import mavutil
import math
import tkinter as tk
import socket
import time
import helper

In [ ]:
SIM_COMPUTER_IP = '192.168.1.124'  # IP address of the simulation computer
PORT = 15000  # The same port as used by the server

In [ ]:
def send_command(command):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((SIM_COMPUTER_IP, PORT))
        s.sendall(command.encode('utf-8'))

def connect_dynamic_port(start_port, max_attempts=10):
    for attempt in range(max_attempts):
        port = start_port + attempt
        try:
            connection = mavutil.mavlink_connection(f'udpin:0.0.0.0:{port}')
            connection.wait_heartbeat()  # Wait for the first heartbeat
            print(f"Connected on UDP port {port}")
            return connection
        except OSError as e:
            if e.errno == 48:  # Address already in use
                print(f"Port {port} is in use, trying next port.")
            else:
                raise
    raise RuntimeError("Failed to connect using dynamic ports.")

def start_instance(instance_id, out_port):
    send_command(f"start {instance_id} {out_port}")

def stop_instance(instance_id):
    send_command(f"stop {instance_id}")


def connect(port):
    connection = mavutil.mavlink_connection(f'udpin:0.0.0.0:{port}') 

    connection.wait_heartbeat() #wait until we hear a heartbeat from the copter

    print("Heartbeat from system (system %u component %u)" % (connection.target_system, connection.target_component))

    return connection 

def get_current_position(connection):

    msg = connection.recv_match(type='GLOBAL_POSITION_INT', blocking=True)

    latitude = msg.lat / 1E7  # Convert from int32 to degrees
    longitude = msg.lon / 1E7  # Convert from int32 to degrees
    altitude = msg.alt / 1000.0  # Convert from millimeters to meters

    return(latitude, longitude, altitude)
    #print(f"Current Position: Latitude: {latitude}, Longitude: {longitude}, Altitude: {altitude} meters")  

def arm(mavlink_connection):
    """
    Arms vehicle and fly to a target altitude.
    :param mavlink_connection: The connection to the vehicle
    :param target_altitude: Target altitude in meters
    """

    print("Basic pre-arm checks")
    # Wait for vehicle to initialize and become ready
    while not mavlink_connection.wait_heartbeat(timeout=5):
        print("Waiting for vehicle heartbeat")

           
    print("Setting vehicle to GUIDED mode")
    mavlink_connection.mav.command_long_send(
        mavlink_connection.target_system, mavlink_connection.target_component,
        mavutil.mavlink.MAV_CMD_DO_SET_MODE,  # Command to set mode
        0,  # Confirmation
        mavutil.mavlink.MAV_MODE_FLAG_CUSTOM_MODE_ENABLED,  # Enable custom modes
        mavutil.mavlink.MAV_MODE_GUIDED_ARMED,  # Set to GUIDED mode
        0, 0, 0, 0, 0, 0  # Other parameters are not used
    )

    # Wait a bit for the mode to change
    time.sleep(2)

    
    print("Arming motors")
    # Copter should arm in GUIDED mode
    mavlink_connection.mav.command_long_send(
        mavlink_connection.target_system, mavlink_connection.target_component,
        mavutil.mavlink.MAV_CMD_COMPONENT_ARM_DISARM,
        0,  # Confirmation
        1,  # 1 to arm
        0, 0, 0, 0, 0, 0  # Unused parameters for this command
    )

In [ ]:
out_port = 14560
pid = 0

while True:
        out_port += 1
        pid += 1

        print(f"starting instance {pid} on port {out_port}")
        start_instance(pid, out_port)
        drone_connection = connect(out_port)


        # Print heartbeat information
        while True:
            msg = drone_connection.recv_match(type='HEARTBEAT', blocking=True)
            if not msg:
                print("No heartbeat")
            else:
                print("Heartbeat received from system (system %u component %u)" % (msg.get_srcSystem(), msg.get_srcComponent()))
                break  # or remove to keep listening

        arm(drone_connection)

        for i in range(20):
            lat, lon, alt = get_current_position(drone_connection)
            print(f'lat: {lat}')
            print(f'lon: {lon}')
            print(f'alt: {alt}')
            
            helper.collect_positions(lat, lon, alt)

        time.sleep(1)

        helper.plot_trajectory()
        helper.reset_trajectory()

        print(f"stopping instance {pid}")
        stop_instance(pid)